In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader,Dataset
import os
import torch.nn.functional as F

In [2]:
#准备数据集
train_dataset=torchvision.datasets.CIFAR10('dataset',train=True,transform=torchvision.transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.CIFAR10('dataset',train=False,transform=torchvision.transforms.ToTensor(),download=True)
print(len(train_dataset))
print(len(test_dataset))

Files already downloaded and verified
Files already downloaded and verified
50000
10000


In [3]:
type(train_dataset)

torchvision.datasets.cifar.CIFAR10

In [4]:
train_dataloader=DataLoader(train_dataset,batch_size=128,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=128,shuffle=True)


In [5]:
train_data_size=len(train_dataset)
test_data_size=len(test_dataset)

In [6]:

'''-------------一、BasicBlock模块-----------------------------'''
# 用于ResNet18和ResNet34基本残差结构块
class BasicBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(BasicBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True), #inplace=True表示进行原地操作，一般默认为False，表示新建一个变量存储操作
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        #论文中模型架构的虚线部分，需要下采样
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
 
    def forward(self, x):
        out = self.left(x) #这是由于残差块需要保留原始输入
        out += self.shortcut(x)#这是ResNet的核心，在输出上叠加了输入x
        out = F.relu(out)
        return out

In [7]:

'''----------ResNet34----------'''
class ResNet_34(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet_34, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 3, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 4, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 6, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 3, stride=2)
        self.fc = nn.Linear(512, num_classes)
 
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
 
    def forward(self, x):  # 3*32*32
        out = self.conv1(x)  # 64*32*32
        out = self.layer1(out)  # 64*32*32
        out = self.layer2(out)  # 128*16*16
        out = self.layer3(out)  # 256*8*8
        out = self.layer4(out)  # 512*4*4
        out = F.avg_pool2d(out, 4)  # 512*1*1
        out = out.view(out.size(0), -1)  # 512
        out = self.fc(out)
        return out

In [8]:
model=ResNet_34(BasicBlock)
model=model.cuda()

In [9]:
print(model)

ResNet_34(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplac

In [10]:
#损失函数和优化器
loss_fn=nn.CrossEntropyLoss()
loss_fn=loss_fn.cuda()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [11]:
total_train_step=0
total_test_step=0

In [12]:
for i in range(30):
    print('第{}轮训练'.format(i+1))
    model.train()
    for data in train_dataloader:
        imgs,targets=data
        imgs=imgs.cuda()
        targets=targets.cuda()
        outputs=model(imgs)
        loss=loss_fn(outputs,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step=total_train_step+1
        if total_train_step%500==0:
            print('训练次数：{},Loss:{}'.format(total_train_step,loss))
    
    model.eval()
    total_test_loss=0
    total_accuracy=0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets=data
            imgs=imgs.cuda()
            targets=targets.cuda()
            outputs=model(imgs)
            loss=loss_fn(outputs,targets)
            total_test_loss=total_test_loss+loss
            accuracy=(outputs.argmax(1)==targets).sum()
            total_accuracy=total_accuracy+accuracy
    print("整体测试集上的Loss:{}".format(total_test_loss))
    print('整体测试集上的正确率:{}'.format(total_accuracy/test_data_size))
    total_test_step=total_test_step+1
    

第1轮训练
整体测试集上的Loss:113.05266571044922
整体测试集上的正确率:0.4851999878883362
第2轮训练
训练次数：500,Loss:1.128670334815979
整体测试集上的Loss:126.02482604980469
整体测试集上的正确率:0.48649999499320984
第3轮训练
训练次数：1000,Loss:0.6516891121864319
整体测试集上的Loss:74.59690856933594
整体测试集上的正确率:0.6796999573707581
第4轮训练
训练次数：1500,Loss:0.5592063069343567
整体测试集上的Loss:68.54907989501953
整体测试集上的正确率:0.7067999839782715
第5轮训练
整体测试集上的Loss:95.30974578857422
整体测试集上的正确率:0.6500999927520752
第6轮训练
训练次数：2000,Loss:0.1519148051738739
整体测试集上的Loss:141.49183654785156
整体测试集上的正确率:0.5999999642372131
第7轮训练
训练次数：2500,Loss:0.16228657960891724
整体测试集上的Loss:93.58724975585938
整体测试集上的正确率:0.6866999864578247
第8轮训练
训练次数：3000,Loss:0.16059668362140656
整体测试集上的Loss:96.00790405273438
整体测试集上的正确率:0.703499972820282
第9轮训练
训练次数：3500,Loss:0.06532320380210876
整体测试集上的Loss:107.36457824707031
整体测试集上的正确率:0.6924999952316284
第10轮训练
整体测试集上的Loss:91.18270874023438
整体测试集上的正确率:0.731499969959259
第11轮训练
训练次数：4000,Loss:0.04160592332482338
整体测试集上的Loss:98.0864028930664
整体测试集上的正确率:0.7292999625205